# Stock Value Prediction

---
## Setup Instructions

Before running this notebook, ensure you have the data ready. You have two options:

1. **Automatic Download**: Run `setup.ipynb`, which will retrieve the data directly from github.
2. **Manual Download**: Retrieve the dataset manually from [this Github repository](https://github.com/dhhagan/stocks/blob/master/scripts/stock_info.csv) and place the csv into the `data/input` folder.

---

## Overview

This notebook explores **training a neural network to predict stock prices** using **PyTorch** and **TurbaNet**, a framework for training multiple models concurrently. Our primary objectives include:

- Training a **baseline PyTorch model** for each stock.
- Training a **swarm of TurbaNet models** on the data.
- Comparing the performance of PyTorch and TurbaNet models in terms of runtime.
- Demonstrating the **effectivity of TurbaNet to train a large number of small models concurrently**, leveraging vectorization when memory constraints allow.

---

## Model Training Approach

#### **Control Model (PyTorch)**
We start with a standard LSTM trained using PyTorch. 
- Each stock will be given its own independent model that PyTorch will train individually. 
- The models will be evaluated based on **loss metrics**.

#### **Swarm-Based Training (TurbaNet)**
Next, we employ TurbaNet to train Jax based LSTM models
- Each stock will be given its own independent model that TurbaNet will train **simultaneously**. 
- This swarm-based approach aims to maximize **computational efficiency** while maintaining model performance.
- The models will be evaluated based on **loss metrics**.

#### **Performance Comparison**
After training, we will:
- Analyze batch-wise loss trends.
- Compare the predictions from both approaches on all stocks.
- Evaluate the real-time efficiency of TurbaNet compared to traditional training methods.

In [ ]:
import time

import jax
import matplotlib.pyplot as plt
import numpy as np
import optax
import pandas as pd
import torch
import torch.nn as nn
import seaborn as sns


from flax import linen
from rich.table import Table
from rich.console import Console
from sklearn.model_selection import train_test_split
from stockdex import Ticker
from torch.utils.data import TensorDataset, DataLoader
from turbanet import TurbaTrainState, mse

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

---
# Hyperparameters

We define key hyperparameters for training our models, here is an example of a configuration:

- **EPOCHS = 10,000** → Train for 10,000 full passes over the dataset.
- **SWARM_SIZE = 100** → Train 100 models on different tickers concurrently using TurbaNet.
- **BATCH_SIZE = 8** → Process 8 sequences per training batch.
- **LR = 1e-5** → Learning rate set to `0.00001` for stable convergence.
- **CPU = True** → Force PyTorch to use the CPU instead of the GPU. 
    - This can be helpful for comparing performance of the libraries when running on Windows machines as [Jax does not support GPUs running on Windows](https://docs.jax.dev/en/latest/installation.html).
- **TIME_WINDOW = 20** → Sequence length for the input prior to the prediction.
- **HIDDEN_SIZE = 32** → Size of the hidden layer of the network.

These settings balance **training stability** and **efficiency**, leveraging **swarm-based training** to speed up model convergence.

Feel free to tweak this values and examine the impact on training speed and effectivity.

In [ ]:
EPOCHS = 1000
SWARM_SIZE = 500
BATCH_SIZE = 32
LR = 1e-5
CPU = False

In [ ]:
TIME_WINDOW = 20
HIDDEN_SIZE = 32

In [ ]:
LOG_FREQUENCY = 5

# Data Collection

In this section, we have implemented the process of collecting, processing, and preparing the stock price data for use in training our model. Here’s what will happen:

- **Stock Data Import**: We loaded the stock ticker information from a CSV file, which contains the tickers of various stocks.
- **Sequence Creation**: We defined a function to generate sequences of stock price data, where each sequence is followed by the next price point, to use as input for time series modeling.
- **Data Retrieval**: Stock data for each ticker is fetched from Yahoo Finance, normalized, and transformed into sequences.
- **Data Filtering**: A random sample of tickers is selected, with validation to ensure that each ticker has sufficient data and consistent sequence lengths.
- **Data Splitting**: The dataset is split into training, validation, and testing sets, ensuring that we reserve the most recent data for testing.
- **PyTorch Data Preparation**: The data is prepared for use with PyTorch by converting it into tensors and creating data loaders for efficient batching during model training.

Next, we will proceed to train models using the prepared data. This will involve building a time series model to predict stock prices based on the historical data we’ve collected.

## Load in Data

In [ ]:
# Load in stock data
df = pd.read_csv("../../data/input/stock_info.csv")
df.head()

## Helper Functions

In [ ]:
# Create sequences of price data of length TIME_WINDOW
def create_sequences(data, time_window) -> tuple[np.ndarray, np.ndarray]:
    sequences = []
    results = []
    for i in range(len(data) - time_window):
        sequence = data[i : i + time_window]
        sequences.append(sequence)
        results.append(data[i + time_window])
    return np.array(sequences).reshape(-1, time_window, 1), np.array(results).reshape(-1, 1)

In [ ]:
def get_ticker_data(ticker, date_range="1y", data_granularity="1d"):
    data = Ticker(ticker=ticker).yahoo_api_price(
        range=date_range, dataGranularity=data_granularity
    )
    close = data["close"]
    if close.empty:
        return None
    close /= np.max(close)
    X_data, y_data = create_sequences(close, TIME_WINDOW)
    return X_data, y_data

In [ ]:
def get_random_ticker_data(stock_df, num_samples, date_range="1y", data_granularity="1d"):
    tickers = []
    X_data = []
    y_data = []
    max_seq_len = 0
    while len(tickers) < num_samples:
        ticker = stock_df.sample(1).Ticker.values[0]
        try:
            ticker_data = get_ticker_data(ticker, date_range, data_granularity)
        except Exception:
            continue

        # If the data returned is None, try again
        if ticker_data is None:
            continue

        if ticker_data[0].shape[0] < max_seq_len:
            continue

        if ticker_data[0].shape[0] > max_seq_len:
            max_seq_len = ticker_data[0].shape[0]
            invalid_tickers = [x.shape[0] < max_seq_len for x in X_data]
            if len(invalid_tickers) > 0:
                del tickers[invalid_tickers]
                del X_data[invalid_tickers]
                del y_data[invalid_tickers]

        tickers.append(ticker)
        X_data.append(ticker_data[0])
        y_data.append(ticker_data[1])

        print(f"Tickers Found: {len(tickers)}/{num_samples}", end="\r", flush=True)

    return tickers, np.array(X_data), np.array(y_data)

## Retrieve Data from Yahoo Finance

In [ ]:
TICKERS, X_data, y_data = get_random_ticker_data(
    df, SWARM_SIZE, date_range="2y", data_granularity="1d"
)

## Display Selected Tickers

In [ ]:
def display_tickers_table(tickers, cols=5):
    table = Table(title="Stock Tickers", show_header=False)
    for i in range(0, len(tickers), cols):
        table.add_row(*tickers[i : i + cols], *[""] * (cols - len(tickers[i : i + cols])))

    console = Console()
    console.print(table)


display_tickers_table(TICKERS, cols=np.sqrt(len(TICKERS)).astype(int))

## Batching and Splitting

In [ ]:
# Create dataset and dataloader
if BATCH_SIZE is None:
    BATCH_SIZE = X_data.shape[1]
X_data = X_data.transpose((1, 0, 2, 3))
y_data = y_data.transpose((1, 0, 2))

In [ ]:
# Reserve most recent 10% of data for testing
train_index = int(X_data.shape[0] * 0.9)
X_train = X_data[:train_index, :, :, :]
y_train = y_data[:train_index, :, :]
X_test = X_data[train_index:, :, :, :]
y_test = y_data[train_index:, :, :]

In [ ]:
# Split data into train, validation, and test sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")

## Dataset and Dataloader

In [ ]:
train_dataset = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
validation_dataset = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))
test_dataset = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)

---
# PyTorch Model Training

In this section, we create **baseline LSTMs** using PyTorch to establish a performance benchmark.

### Training Details:
- **Loss Function**: Cross-Entropy Loss
- **Optimizer**: Adam

This PyTorch model serves as a **control experiment**, helping us compare its efficiency and accuracy against the **swarm-based training** approach used in TurbaNet.


In [ ]:
## Checking if the GPU is being used properly.
DEVICE = torch.device("cuda" if torch.cuda.is_available() and not CPU else "cpu")
print("Using device:", DEVICE)

In [ ]:
class TorchLSTM(nn.Module):
    def __init__(self, hidden_size=128):
        super(TorchLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(1, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # Initialize hidden and cell state (h_0, c_0) with zeros
        batch_size = x.size(0)
        h_0 = torch.zeros(1, batch_size, self.hidden_size, dtype=torch.float).to(x.device)
        c_0 = torch.zeros(1, batch_size, self.hidden_size, dtype=torch.float).to(x.device)

        # Forward propagate LSTM
        out, (h_n, c_n) = self.lstm(x, (h_0, c_0))  # Shape: (batch, seq_len, hidden_size)

        # Take the last time step output
        final_output = out[:, -1, :]  # Shape: (batch, hidden_size)

        # Fully connected layer to map hidden state to final output
        return self.fc(final_output)  # Shape: (batch, output_size)

In [ ]:
torch_models = [TorchLSTM(hidden_size=HIDDEN_SIZE).to(DEVICE) for _ in range(SWARM_SIZE)]
torch_optimizers = [torch.optim.Adam(model.parameters(), lr=LR) for model in torch_models]
torch_loss = torch.nn.MSELoss().to(DEVICE)

In [ ]:
def torch_train(dataloader, models, optimizers):
    losses = np.empty((0, SWARM_SIZE))
    for batch, (X, y) in enumerate(dataloader):
        X = torch.transpose(X, 1, 0).type(torch.float32).to(DEVICE)
        y = torch.transpose(y, 1, 0).type(torch.float32).to(DEVICE)

        # Train
        losses = np.vstack((losses, np.zeros(SWARM_SIZE)))
        for idx, (model, optimizer) in enumerate(zip(models, optimizers)):
            y_pred = model(X[idx])
            loss = torch_loss(y_pred, y[idx])

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            losses[batch, idx] = loss.item()

        # Logging
        if batch % LOG_FREQUENCY == 0:
            # Display the first 5 model's losses (or less if less than 5)
            loss = losses[-1]
            current = batch * X[0].shape[0]
            loss_string = ", ".join([f"{loss[i]:.4f}" for i in range(min(5, SWARM_SIZE))])
            print(f"loss: {loss_string}  [{current:>5d}]")

    return losses

In [ ]:
def torch_validation(dataloader, models):
    losses = np.empty((0, SWARM_SIZE))
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X = torch.transpose(X, 1, 0).type(torch.float32).to(DEVICE)
            y = torch.transpose(y, 1, 0).type(torch.float32).to(DEVICE)

            # Train
            losses = np.vstack((losses, np.zeros(SWARM_SIZE)))
            for idx, model in enumerate(models):
                y_pred = model(X[idx])
                loss = torch_loss(y_pred, y[idx])
                losses[batch, idx] = loss.item()

    # Logging
    losses = losses.mean(axis=0)
    loss_string = ", ".join([f"{losses[i]:.4f}" for i in range(min(5, SWARM_SIZE))])
    print(f"Validation loss: {loss_string}")

    return losses

In [ ]:
# Train the PyTorch model
start = time.time()
train_epoch_no = []
train_batch_loss = []
valid_batch_loss = np.empty((0, SWARM_SIZE))

for t in range(EPOCHS):
    print(f"\nEpoch {t + 1}\n-------------------------------")
    # Training
    for torch_model in torch_models:
        torch_model.train()
    _train_batch_losses = torch_train(train_dataloader, torch_models, torch_optimizers)

    # Validation
    for torch_model in torch_models:
        torch_model.eval()
    _valid_batch_losses = torch_validation(validation_dataloader, torch_models)

    # Train data
    for i in range(len(_train_batch_losses)):
        train_epoch_no.append(t + float((i + 1) / len(_train_batch_losses)))
        train_batch_loss.append(_train_batch_losses[i])

    # Validation data
    valid_batch_loss = np.vstack((valid_batch_loss, _valid_batch_losses))

torch_time = time.time() - start
print(f"torch time: {torch_time}")

In [ ]:
train_batch_loss = np.array(train_batch_loss)
valid_batch_loss = np.array(valid_batch_loss)

In [ ]:
# Plot of losses over training and validation
sqrt_size = int(np.ceil(np.sqrt(SWARM_SIZE)))
fig = plt.figure(figsize=(5 * sqrt_size, 5 * sqrt_size))
for idx, ticker in enumerate(TICKERS):
    fig.add_subplot(int(np.ceil(np.sqrt(SWARM_SIZE))), int(np.ceil(np.sqrt(SWARM_SIZE))), idx + 1)
    plt.plot(train_epoch_no, train_batch_loss[:, idx], label=ticker)
    plt.plot(np.arange(1, EPOCHS + 1), valid_batch_loss[:, idx], label=f"Validation {ticker}")

    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title(TICKERS[idx] + " Loss")
    plt.legend()

plt.show()


In [ ]:
torch_predictions = []
for idx, torch_model in enumerate(torch_models):
    torch_model.eval()
    X = torch.Tensor(X_test.transpose((1, 0, 2, 3))[0]).to(DEVICE)
    torch_predictions.append(torch_model(X).cpu().detach().numpy())

In [ ]:
y_true = np.empty((0, SWARM_SIZE))
torch_predictions = np.empty((0, SWARM_SIZE))
for batch, (X, y) in enumerate(test_dataloader):
    X = torch.transpose(X, 1, 0).type(torch.float32).to(DEVICE)
    y = torch.transpose(y, 1, 0).type(torch.float32).to(DEVICE)

    # Evaluate each model
    y_true = np.vstack((y_true, y.reshape((SWARM_SIZE, X.shape[1])).transpose(1, 0).cpu()))
    predictions = np.empty((X.shape[1], SWARM_SIZE))
    for idx, model in enumerate(torch_models):
        model.eval()

        y_pred = model(X[idx])
        predictions[:, idx] = y_pred.cpu().detach().numpy().T

    torch_predictions = np.vstack((torch_predictions, predictions))

In [ ]:
# Subplot of predictions vs ground truth (x by x)
fig = plt.figure(figsize=(5 * sqrt_size, 5 * sqrt_size))

# Shared axes
for i in range(SWARM_SIZE):
    ax = fig.add_subplot(
        int(np.ceil(np.sqrt(SWARM_SIZE))), int(np.ceil(np.sqrt(SWARM_SIZE))), i + 1
    )
    ax.plot(y_true[:, i], label="Ground Truth")
    ax.plot(torch_predictions[:, i], label="Torch Prediction")
    ax.set_title(TICKERS[i])
    ax.set_ylabel("Price")
    ax.legend(loc="upper left")

plt.show()

# Turba

In [ ]:
torch.cuda.empty_cache()

In [ ]:
class TurbaLSTM(linen.Module):
    features: int

    @linen.compact
    def __call__(self, x):
        ScanLSTM = linen.scan(
            linen.OptimizedLSTMCell,
            variable_broadcast="params",
            split_rngs={"params": False},
            in_axes=1,
            out_axes=1,
        )

        lstm = ScanLSTM(self.features)
        input_shape = x[:, 0].shape
        carry = lstm.initialize_carry(jax.random.PRNGKey(0), input_shape)
        carry, x = lstm(carry, x)
        final = x[:, -1]
        output = linen.Dense(1)(final)
        return output

In [ ]:
optimizer = optax.adam(learning_rate=LR)

In [ ]:
turba_model = TurbaTrainState.swarm(
    TurbaLSTM(features=HIDDEN_SIZE),
    optimizer,
    SWARM_SIZE,
    X_data[0][0].reshape((1, TIME_WINDOW, 1)),
)

In [ ]:
def turba_train(dataloader: DataLoader, models: TurbaTrainState):
    losses = np.empty((0, SWARM_SIZE))
    for batch, (X, y) in enumerate(dataloader):
        X = torch.transpose(X, 1, 0)
        y = torch.transpose(y, 1, 0)

        # Train
        models, loss, _ = models.train(X, y, mse)
        losses = np.vstack((losses, loss))

        # Logging
        if batch % LOG_FREQUENCY == 0:
            # Display the first 5 model's losses (or less if less than 5)
            loss = losses[-1]
            current = batch * X[0].shape[0]
            loss_string = ", ".join([f"{loss[i]:.4f}" for i in range(min(5, SWARM_SIZE))])
            print(f"loss: {loss_string}  [{current:>5d}]")

    return models, losses

In [ ]:
def turba_validation(dataloader: DataLoader, models: TurbaTrainState):
    losses = np.empty((0, SWARM_SIZE))
    y_true = np.empty((0, SWARM_SIZE))
    predictions = np.empty((0, SWARM_SIZE))
    for batch, (X, y) in enumerate(dataloader):
        X = torch.transpose(X, 1, 0).numpy()
        y = torch.transpose(y, 1, 0).numpy()

        y_true = np.vstack((y_true, y.reshape((SWARM_SIZE, X.shape[1])).transpose(1, 0)))

        # Train
        loss, y_pred = models.evaluate(X, y, mse)
        losses = np.vstack((losses, loss))
        predictions = np.vstack(
            (predictions, y_pred.reshape((SWARM_SIZE, X.shape[1])).transpose((1, 0)))
        )

    # Display the first 5 model's losses (or less if less than 5)
    loss = losses[-1]
    current = batch * X[0].shape[0]
    loss_string = ", ".join([f"{loss[i]:.4f}" for i in range(min(5, SWARM_SIZE))])
    print(f"loss: {loss_string}  [{current:>5d}]")

    return losses.mean(axis=0), predictions, y_true

In [ ]:
# Train the Turba model
start = time.time()
train_epoch_no = []
train_batch_loss = []
valid_batch_loss = np.empty((0, SWARM_SIZE))

for t in range(EPOCHS):
    print(f"\nEpoch {t + 1}\n-------------------------------")
    # Training
    turba_model, _train_batch_losses = turba_train(train_dataloader, turba_model)

    # Validation
    _valid_batch_losses, _, _ = turba_validation(validation_dataloader, turba_model)

    # Train data
    for i in range(len(_train_batch_losses)):
        train_epoch_no.append(t + float((i + 1) / len(_train_batch_losses)))
        train_batch_loss.append(_train_batch_losses[i])

    # Validation data
    valid_batch_loss = np.vstack((valid_batch_loss, _valid_batch_losses))

turba_time = time.time() - start
print(f"turba time: {turba_time}")

In [ ]:
train_batch_loss = np.array(train_batch_loss)
valid_batch_loss = np.array(valid_batch_loss)

In [ ]:
# Plot of losses over training and validation
sqrt_size = int(np.ceil(np.sqrt(SWARM_SIZE)))
fig = plt.figure(figsize=(5 * sqrt_size, 5 * sqrt_size))
for idx, ticker in enumerate(TICKERS):
    fig.add_subplot(int(np.ceil(np.sqrt(SWARM_SIZE))), int(np.ceil(np.sqrt(SWARM_SIZE))), idx + 1)
    plt.plot(train_epoch_no, train_batch_loss[:, idx], label=ticker)
    plt.plot(np.arange(1, EPOCHS + 1), valid_batch_loss[:, idx], label=f"Validation {ticker}")

    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title(TICKERS[idx] + " Loss")
    plt.legend()

plt.show()


In [ ]:
y_true = np.empty((0, SWARM_SIZE))
turba_predictions = np.empty((0, SWARM_SIZE))
for batch, (X, y) in enumerate(test_dataloader):
    X = torch.transpose(X, 1, 0).numpy()
    y = torch.transpose(y, 1, 0).numpy()

    y_true = np.vstack((y_true, y.reshape((SWARM_SIZE, X.shape[1])).transpose(1, 0)))

    # Train
    y_pred = turba_model.predict(X)
    turba_predictions = np.vstack(
        (turba_predictions, y_pred.reshape((SWARM_SIZE, X.shape[1])).transpose((1, 0)))
    )


In [ ]:
# Subplot of predictions vs ground truth (x by x)
fig = plt.figure(figsize=(5 * sqrt_size, 5 * sqrt_size))

# Shared axes
for i in range(SWARM_SIZE):
    ax = fig.add_subplot(
        int(np.ceil(np.sqrt(SWARM_SIZE))), int(np.ceil(np.sqrt(SWARM_SIZE))), i + 1
    )

    # Title
    ax.set_title(TICKERS[i])

    # Axes
    ax.set_ylabel("Price")

    # Data
    ax.plot(y_true[:, i], label="Ground Truth")
    ax.plot(turba_predictions[:, i], label="Turba Prediction")

    # Legend
    ax.legend(loc="upper left")

plt.show()


# Results Comparison

In [ ]:
# Subplot of predictions vs ground truth (x by x)
fig = plt.figure(figsize=(5 * sqrt_size, 5 * sqrt_size))

# Shared axes
for i in range(SWARM_SIZE):
    ax = fig.add_subplot(
        int(np.ceil(np.sqrt(SWARM_SIZE))), int(np.ceil(np.sqrt(SWARM_SIZE))), i + 1
    )

    # Title
    ax.set_title(TICKERS[i])

    # Axes
    ax.set_ylabel("Price")

    # Data
    ax.plot(y_true[:, i], label="Ground Truth")
    ax.plot(torch_predictions[:, i], label="Torch Prediction")
    ax.plot(turba_predictions[:, i], label="Turba Prediction")

    # Legend
    ax.legend(loc="upper left")

plt.show()

In [ ]:
# Calculate the average error for each model
error_torch = np.mean(np.abs(y_true - torch_predictions), axis=0)
error_turba = np.mean(np.abs(y_true - turba_predictions), axis=0)
error_torch = error_torch[~np.isnan(error_torch)]
error_turba = error_turba[~np.isnan(error_turba)]

# Combine both error arrays to calculate common bin edges
combined_errors = np.concatenate([error_torch, error_turba])

# Define the number of bins you want (e.g., 30 bins)
num_bins = 30

# Calculate bin edges based on combined data
bin_edges = np.linspace(np.min(combined_errors), np.max(combined_errors), num_bins + 1)

# Set the seaborn style for a polished look
sns.set_theme(context="notebook", style="whitegrid", palette="tab10")

# Plotting the KDE with fill
plt.figure(figsize=(12, 8))

# Plot histogram for Torch model using the common bin edges
sns.histplot(
    error_torch,
    kde=True,
    stat="probability",
    fill=True,
    alpha=0.6,
    linewidth=2,
    bins=bin_edges,  # Use the predefined bin edges
    label="Torch Models",
)

# Plot histogram for Turba model using the common bin edges
sns.histplot(
    error_turba,
    kde=True,
    stat="probability",
    fill=True,
    alpha=0.6,
    linewidth=2,
    bins=bin_edges,  # Use the predefined bin edges
    label="Turba Models",
)

# Add a title with a larger, bold font
plt.title("Comparison of Average Errors: Torch vs. Turba", fontsize=18, weight="bold")

# Add labels with larger, bolder fonts
plt.xlabel("Average Error", fontsize=15, weight="bold")
plt.ylabel("Probability", fontsize=15, weight="bold")

# Increase the font size of the legend
plt.legend(fontsize=14, title="Models", title_fontsize="13", loc="upper right")

# Refine the axes with ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Remove the top and right borders for a cleaner look
sns.despine()

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
from IPython.core.display import display_functions, HTML

output = """
===========================
Model       Training Time
===========================
PyTorch       {:.2f} sec
---------------------------
Turba         {:.2f} sec
===========================
""".format(torch_time, turba_time)

display_functions.display(HTML(f"<pre>{output}</pre>"))